<a id='Title-District-Prices'></a>
<h1 style="color:SlateGray;">District Prices</h1>

**Input dataset**

California Housing Prices: a set of 9 characteristics of the house such as location, age, etc.

**Output classification**

Estimate of average house prices by district.

<h2 style="color:SlateGray;">Background</h2>

**Models**

<a id='Models-LinearRegression'></a>
*LinearRegression*

Models the linear relationship between data categories from the dataset.

Approximates the relationship between data categories with linear equations of the form $\textbf{y}_i = \theta_1 \cdot \textbf{x}_j + \theta_2$.

<a id='Models-DecisionTreeRegressor'></a>
*DecisionTreeRegressor*

Models non-linear relationships between data categories with a binary tree.

Divides the graph of all data points with segments along the axes which box around subsets of points forming a prediction. Tree traversal is achieved by checking if an input value is above or below the threshold set by the axis segments. 

<a id='Models-RandomForestRegressor'></a>
*RandomForestRegressor*

A single randomly initialized [*decision tree's*](#Models-DecisionTreeRegressor) predictions will be biased, thus, using large numbers of these biased trees will statistically converge around a more accurate prediction.

Spawns a specified number of [*decision trees*](#Models-DecisionTreeRegressor) and tallies up each tree's predictions, the prediction with the highest number of votes is used as the model's final prediction. Trees with low prediction scores are replaced by new randomly configured trees.

<h2 style="color:SlateGray;">Overview</h2>

<img src='PlantUML/01_District_Prices.svg?sanitize=true'>

<h2 style="color:SteelBlue;">Loading the data</h2>

In [1]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = r'https://raw.githubusercontent.com/ageron/handson-ml/master/'
HOUSING_PATH = r'datasets/housing'
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + r'/housing.tgz'

print(HOUSING_URL)

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
        
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)
    
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz


In [2]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

<h2 style="color:SteelBlue;">Creating training and testing sets</h2>

In [3]:
import numpy as np

housing = load_housing_data()
housing['income_catagory'] = np.ceil(housing['median_income'] / 1.5)
housing['income_catagory'].where(housing['income_catagory'] < 5, 5.0, inplace=True)

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['income_catagory']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [5]:
for set in (strat_train_set, strat_test_set):
    set.drop(['income_catagory'], axis=1, inplace=True)

<h2 style="color:SteelBlue;">Preparing the data</h2>

In [6]:
housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

housing_num = housing.drop('ocean_proximity', axis=1)
housing_cat = housing['ocean_proximity']

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([('imputer', Imputer(strategy='median')),
                         ('attribs_adder', CombinedAttributesAdder()),
                         ('std_scaler', StandardScaler())])

c:\users\manny\anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [9]:
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([('num', num_pipeline, num_attribs),
                                   ('cat', OneHotEncoder(), cat_attribs)])
housing_prepared = full_pipeline.fit_transform(housing)

c:\users\manny\anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


<h2 style="color:SteelBlue;">Training and evaluating on the training set</h2>

In [10]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

In [11]:
def display_training_set_performance(model, rmse):
    print(f'RMSE: {rmse}')
    print(f'Predictions: {model.predict(some_data_prepared)}')
    print(f'Labels: {list(some_labels)}')

In [12]:
def display_scores(scores):
    print(f'Scores: {scores}')
    print(f'Mean: {scores.mean()}')
    print(f'Standard deviation: {scores.std()}')

<h3 style="color:LightSlateGray;">Linear Regression</h3>

In [13]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [14]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
line_rmse = np.sqrt(lin_mse)
display_training_set_performance(lin_reg, line_rmse)

RMSE: 68628.19819848923
Predictions: [210644.60459286 317768.80697211 210956.43331178  59218.98886849
 189747.55849879]
Labels: [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


In [15]:
from sklearn.model_selection import cross_val_score

lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, 
                             scoring='neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [66782.73843989 66960.118071   70347.95244419 74739.57052552
 68031.13388938 71193.84183426 64969.63056405 68281.61137997
 71552.91566558 67665.10082067]
Mean: 69052.46136345083
Standard deviation: 2731.6740017983425


<h3 style="color:LightSlateGray;">Decision Tree Regressor</h3>

In [16]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [17]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
display_training_set_performance(tree_reg, tree_rmse)

RMSE: 0.0
Predictions: [286600. 340600. 196900.  46300. 254500.]
Labels: [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


In [18]:
tree_scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                              scoring='neg_mean_squared_error', cv=10)
tree_rmse_scores = np.sqrt(-tree_scores)
display_scores(tree_rmse_scores)

Scores: [68913.17508049 65601.47438473 71774.90639524 69732.73417516
 71195.97233499 75031.69189519 69887.11659569 70719.67518527
 78081.34461047 70199.2274029 ]
Mean: 71113.73180601337
Standard deviation: 3224.4801880340106


<h3 style="color:LightSlateGray;">Random Forest Regressor</h3>

In [19]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10)
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [20]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
display_training_set_performance(forest_reg, forest_rmse)

RMSE: 22464.884941038657
Predictions: [265470. 309960. 214460.  53570. 239940.]
Labels: [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


In [21]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, 
                                scoring='neg_mean_squared_error', cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [52791.31598842 48675.83109387 51728.82261755 54139.66959173
 52569.81348754 56384.37862819 50775.04573484 49766.95852751
 56014.34227822 52611.66001201]
Mean: 52545.78379598788
Standard deviation: 2363.7412285915457


<h2 style="color:SteelBlue;">Fine-tune the best model</h2>

In [22]:
from sklearn.model_selection import GridSearchCV

param_grid = [{'n_estimators' : range(5, 50, 5), 
               'max_features' : range(2, 10, 2)}, 
              {'bootstrap' : [False], 
               'n_estimators' : range(5, 50, 5), 
               'max_features' : range(2, 10, 2)}]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, 
                           cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': range(5, 50, 5), 'max_features': range(2, 10, 2)}, {'bootstrap': [False], 'n_estimators': range(5, 50, 5), 'max_features': range(2, 10, 2)}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [23]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 6, 'n_estimators': 45}

In [24]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features=6, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=45, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

<h2 style="color:SteelBlue;">Final model</h2>

In [25]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value'].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [26]:
final_rmse

46942.18787326579

<h2 style="color:SteelBlue;">Confidence interval computation</h2>

In [27]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
mean = squared_errors.mean()
m = len(squared_errors)

np.sqrt(stats.t.interval(confidence, (m-1), 
                         loc=np.mean(squared_errors), 
                         scale=stats.sem(squared_errors)))

array([44908.90897383, 48890.9797349 ])